In [1]:
from src.data import OxfordPetDatasetLoader
from src.config import Config
from src.data import DataPreprocessor, DataAugmentor


config = Config()
config.USE_IMAGENET_NORM = True          # For transfer learning
config.PRESERVE_ASPECT_RATIO = True      # Better image quality
config.ENABLE_QUALITY_ENHANCEMENT = True # Image enhancement
config.NORMALIZATION_METHOD = 'imagenet' # Best for pretrained models

2025-06-17 02:10:57.197215: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 02:10:57.205328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750101057.214777   35508 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750101057.217418   35508 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750101057.224638   35508 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:

loader = OxfordPetDatasetLoader(
    data_dir="./data",  # Tùy chọn
    download=True,
    log_level='INFO'
)

INFO:src.data.dataset_loader:Initialized Oxford Pet Dataset Loader
INFO:src.data.dataset_loader:Data directory: ./data


In [3]:

train_ds, val_ds, test_ds = loader.create_train_val_test_splits(
    val_split=0.2,
    seed=42
)

INFO:src.data.dataset_loader:Creating train/validation/test splits...
INFO:src.data.dataset_loader:Checking dataset availability...
INFO:absl:Load dataset info from data/oxford_iiit_pet/4.0.0
INFO:src.data.dataset_loader:Dataset already available.
INFO:src.data.dataset_loader:Loading raw dataset splits: ['train', 'test']
INFO:absl:Load dataset info from data/oxford_iiit_pet/4.0.0
INFO:absl:Creating a tf.data.Dataset reading 4 files located in folders: data/oxford_iiit_pet/4.0.0.
INFO:absl:Creating a tf.data.Dataset reading 4 files located in folders: data/oxford_iiit_pet/4.0.0.
INFO:absl:Constructing tf.data.Dataset oxford_iiit_pet for split ['train', 'test'], from data/oxford_iiit_pet/4.0.0
INFO:src.data.dataset_loader:Train set: 3,680 examples
INFO:src.data.dataset_loader:Test set: 3,669 examples
INFO:src.data.dataset_loader:Split sizes - Train: 2,944, Val: 736, Test: 3,669


In [4]:
train_ds.element_spec

{'file_name': TensorSpec(shape=(), dtype=tf.string, name=None),
 'head_bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None),
 'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None),
 'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'segmentation_mask': TensorSpec(shape=(None, None, 1), dtype=tf.uint8, name=None),
 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}

In [5]:
augmentor = DataAugmentor(
    config=config,
    prob_geo=0.7,
    prob_photo=0.8,
    prob_mixup=0.3,
    prob_cutout=0.2,
    prob_mosaic=0.3,
)

train_ds = augmentor.create_augmented_dataset(
    train_ds,
    augmentation_factor=3
)

val_ds = augmentor.create_augmented_dataset(
    val_ds,
    augmentation_factor=1
)

In [6]:
train_ds.element_spec

{'image': TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None),
 'head_bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None),
 'segmentation_mask': TensorSpec(shape=(224, 224, 1), dtype=tf.float32, name=None),
 'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'species': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'file_name': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [7]:
processor = DataPreprocessor(
    config=Config(),
    shuffle_buffer=5000,
)

In [8]:
train_ds = processor.create_training_dataset(
    train_ds,
    batch_size=8,
    task="detection",
    cache_filename="train_cache"  # File caching for speed
)

val_ds = processor.create_validation_dataset(
    val_ds,
    batch_size=8,
    task="detection"
)

In [9]:
train_ds.element_spec

(TensorSpec(shape=(8, 224, 224, 3), dtype=tf.float32, name=None),
 {'bbox': TensorSpec(shape=(8, 4), dtype=tf.float32, name=None),
  'pet_class': TensorSpec(shape=(8,), dtype=tf.int32, name=None),
  'species': TensorSpec(shape=(8,), dtype=tf.int32, name=None),
  'valid_bbox': TensorSpec(shape=(8,), dtype=tf.bool, name=None),
  'area': TensorSpec(shape=(8,), dtype=tf.float32, name=None)})

In [10]:
from src.models.base_model import ModelBuilder
from src.config.config import Config


model = ModelBuilder.build_detection_model(
    model_type="pretrained",
    num_classes=37,
    config=Config(),
)

In [11]:
from src.training.trainer import Trainer
from src.config.config import Config
from src.config.model_configs import ModelConfigs
from pathlib import Path

config = Config()
config.CHECKPOINT_DIR = Path("./checkpoints")


trainer = Trainer(
    model=model,
    task_type="detection",
    backbone_name="resnet50",
    config=config,
    models_config=ModelConfigs(),
)

2025-06-17 02:11:05,048 - trainer_135924244493072 - WARNING - Memory growth setting failed: Physical devices cannot be modified after being initialized


In [12]:
trainer.fit(train_ds, val_dataset=val_ds, epochs=10)

2025-06-17 02:11:05.901742: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-06-17 02:11:15.360725: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] fused(ShuffleDatasetV3:20,RepeatDataset:21): Filling up shuffle buffer (this may take a while): 1174 of 5000
2025-06-17 02:11:25.366062: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] fused(ShuffleDatasetV3:20,RepeatDataset:21): Filling up shuffle buffer (this may take a while): 2470 of 5000
2025-06-17 02:11:43.908579: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
I0000 00:00:1750101107.127187   35602 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-06-17 02:11:50,037 - trainer_135924244493072 - ERROR - Error in training step 0: No gradients computed. This usually means:
1. The loss is not connected to model variables
2. The model is not being called properly in train_s

ValueError: No gradients computed. This usually means:
1. The loss is not connected to model variables
2. The model is not being called properly in train_step()
3. Variables are not marked as trainable
Model has 228 trainable variables